#### 1. 시스템 환경 설정

In [ ]:
# DNS등록 : Cloudflare (2건)
# MAC등록 : Firewall (DHCP Reserve : AIBIZ_RND7, Server Group Access)
# DHCP IP 다시 받아오기 (주의 : 네트워크 끊길 수 있음)
nohup bash -c 'sleep 3; sudo ip link set ens22f0 up' & sudo ip link set ens22f0 down

In [ ]:
### VSCode SSH 설정 ###
# ssh 연결 설정
vi /home/aibiz/.ssh/authorized_keys
# Key for aibiz
ssh-rsa AAAAB3NzaC1yc2EAAAABIwAAAgEA38siiI0/EdhQNULtEYvWuTLJRNEzE3jV9SUflv3pZeaVFdoijspD+DfRAYe5dB4oUfQ9mO133WW6ijxqA6Im4dX/Bt/penhWZs4LjcpVH105DuZXXESUALRoR0CNgFtLc7dkAkWdZ1VYL7lX2Z2J+VA5EcYBbSFs+AKfwC4vqRnp0aPAbhsGPq5Vnd2ebL4Cyg+Vk6+SfRuz3a9Ccv+WmtTjMxkTK+bLAMe+0dU3lHr2tNNKk48upBX3JMoJos1TR1k/QkpWfFFQFRrwezYYV6klLfkXARcG/3lZ8em9xXFa4U4tv+XHTvZ5jIEEITaLaqF5fONEm6NvffIELWGIllsljNo2HZhdGtCj+GXo3bHMACuM33sM7NhukaQzj/WcClx1b6LKKmNYjGTTKH35l9E4tQVNPDU969unSnViu8eG4N/fKm4+HPahQIU5DJb0oB9+1yXHGETiyDI3SnKSofktcV2PwijPjH+xBDWh/OmYRLvXyMfXdOPZhk7fZxM0/7j3B/5lAhJeRPJUiHNCVeEbPPcaYqL19iaQKUCxx0bgkPhXNo5XIw6v2Zvh7vwNWoY0CTdvBJK/mD8cHaVBi2ExPIiySGaX8+x55qzSMB3+jYt6TpMaCJOPSINB4nE4Oddj0OtG3M8lZzma+LJJJPTR0yozgnZQs4Bj7e/lVMU=

In [ ]:
### IP : DHCP, DNS: GOOGLE 설정 ###
# 설정파일 편집
sudo vi /etc/netplan/70-netplan-set.yaml
network:
  version: 2
  ethernets:
    ens22f0:
      dhcp4: true
      nameservers:
        addresses:
          - 8.8.8.8
          - 8.8.8.4
        search: [ai-biz.net]


# 변경적용
sudo netplan apply

# 상태확인
resolvectl status

In [ ]:
### 설정 소스 GitLab에서 가져오기 ###
# 프로젝트 폴더로 이동
cd /home/aibiz/setup

# 로컬 Git 저장소 초기화
git init

# GitLab 글로벌 환경 지정
git config --global user.name "jslee"
git config --global user.email "jslee@ai-biz.net"
git config --global credential.helper store

# GitLab 원격 저장소를 추가
git remote add origin https://jslee@gitlab.ai-biz.net/jslee/infra_setup.git
# GitLab 원격 저장소를 변경
git remote set-url origin https://jslee@gitlab.ai-biz.net/jslee/infra_setup.git

# GitLab 최종소스 가져오기
# git pull origin main --allow-unrelated-histories --no-rebase

# 로컬 변경 무시하고 모두 가져오기
git fetch --all
git checkout main

#git reset --hard origin/main

In [ ]:
### 프로젝트(고객사별) 폴더 생성 및 마운트 ###
# 프로젝트 볼륨 생성
sudo mkdir /data/volumes
sudo chown -R aibiz:aibiz /data/volumes
 
cd /home/aibiz/setup
ln -s /data/volumes volumes

# rnd6번 서버에서 전체 복사(11GB)
# scp -r -C aibiz@rnd6:/data/volumes/* /data/volumes/
rsync -avz --progress aibiz@rnd6:/data/volumes/ /data/volumes/
rsync -avz --progress --exclude 'airflow/home/aibiz/ftp-files/' --exclude 'ftp/' --exclude 'dbstdf/' --exclude 'certbot/' --exclude 'jupyterhub/home/aibiz/notebooks/jcet/' aibiz@rnd6:/data/volumes/ /data/volumes/


# 링크는 수동으로 걸어줌
ln -s /container/service/:ssl-tools/assets/default-ca/default-ca.pem /data/volumes/openldap/certificates/ai-biz.net.ca.crt

In [ ]:
### 시스템 환경 설정 ###
# 참고 : VSCode 서비스를 재기동 해야 적용됨.
#    Ctrl + Shift + P => Remote-SSH: Kill VS Code Server on Host...
#    재 접속

# 시스템 환경 (모든 사용자에 적용)
sudo cp ./assets/environment /etc/environment

In [ ]:
### apt 패키지 저장소 설정 ###
# sudo vi /etc/apt/sources.list.d/ubuntu.sources
Types: deb
URIs: https://nexus.ai-biz.net/repository/apt-repos/
Suites: noble noble-updates noble-backports
Components: main restricted universe multiverse
Signed-By: /usr/share/keyrings/ubuntu-archive-keyring.gpg

Types: deb
URIs: http://archive.ubuntu.com/ubuntu/
Suites: noble noble-updates noble-backports
Components: main restricted universe multiverse
Signed-By: /usr/share/keyrings/ubuntu-archive-keyring.gpg

Types: deb
URIs: http://security.ubuntu.com/ubuntu/
Suites: noble-security
Components: main restricted universe multiverse
Signed-By: /usr/share/keyrings/ubuntu-archive-keyring.gpg

In [ ]:
### docker 저장소 설정 ###
# sudo mkdir /etc/docker
# sudo vi /etc/docker/daemon.json
{
    "ipv6": false,
    "insecure-registries": [
        "nexus.ai-biz.net:5000",
        "nexus.ai-biz.net:5001"
    ],
    "registry-mirrors": [
        "http://nexus.ai-biz.net:5000"
    ]
}

In [ ]:
### pip 저장소 설정 ###
# sudo vi /etc/pip.conf
[global]
# 패키지를 다운로드할 때 기본적으로 사용하는 index-url
index-url = https://nexus.ai-biz.net/repository/pypi-repos/simple
trusted-host = nexus.ai-biz.net

# 타임아웃 설정 (초)
timeout = 60

In [ ]:
### Nvidia 드라이버 및 CUDA설치 확인 ###
nvidia-smi
nvcc --version

# (참고) Nvidia 드라이버 설치(미설치된 경우)
sudo apt update
sudo apt upgrade -y
sudo apt install -y build-essential dkms
sudo apt install -y nvidia-driver-560
sudo reboot

# CUDA (12.6) 설치
cd ~/downloads
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/cuda-ubuntu2404.pin
sudo mv cuda-ubuntu2404.pin /etc/apt/preferences.d/cuda-repository-pin-600
wget https://developer.download.nvidia.com/compute/cuda/12.6.3/local_installers/cuda-repo-ubuntu2404-12-6-local_12.6.3-560.35.05-1_amd64.deb
sudo dpkg -i cuda-repo-ubuntu2404-12-6-local_12.6.3-560.35.05-1_amd64.deb
sudo cp /var/cuda-repo-ubuntu2404-12-6-local/cuda-*-keyring.gpg /usr/share/keyrings/
sudo apt-get update
sudo apt-get -y install cuda-toolkit-12-6

# 환경변수 LD_LIBRARY_PATH=/usr/local/cuda-12.6/lib64 걸려 있어야 확인 가능함.

# 정상 저장소 재등록
curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | \
  sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg

curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
  sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
  sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list

# nvidia-docker2 설치
sudo apt-get update
sudo apt-get install -y nvidia-container-toolkit
sudo systemctl restart docker

# Docker가 GPU 접근 가능한지 확인
docker run --rm --gpus all nvidia/cuda:12.2.0-base-ubuntu22.04 nvidia-smi

#### 2. Docker 설치

In [ ]:
# 패키지 관리자 업데이트
sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg
# Docker GPG 키 추가
sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --batch --yes --dearmor -o /etc/apt/keyrings/docker.gpg
sudo chmod a+r /etc/apt/keyrings/docker.gpg
# Docker 리포지토리 설정
echo "deb [arch=`dpkg --print-architecture` signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu `lsb_release -cs` stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
cat /etc/apt/sources.list.d/docker.list
# Docker(28.0.4) 설치
sudo apt-get update
sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin
# Docker 설치 확인
docker --version
# 그룹에 사용자 추가
sudo usermod -aG docker $(whoami)
newgrp docker
# 권한 변경
sudo chown -R aibiz:aibiz /opt/containerd
# Docker Compose(083f676) 설치
sudo curl -L "https://github.com/docker/compose/releases/latest/download/docker-compose-`uname -s`-`uname -m`" -o /usr/local/bin/docker-compose
# Compose 실행 가능하도록 설정
sudo chmod +x /usr/local/bin/docker-compose
# Compose 설치 확인(2.34.0)
docker-compose --version
# 서비스 확인
sudo systemctl status docker --no-pager

#### 3. Spark 설치

In [ ]:
# 폴더로 이동
mkdir -p ~/downloads
cd ~/downloads

# 타임존 지정		
echo "Asia/Seoul" | sudo tee /etc/timezone
sudo ln -sf /usr/share/zoneinfo/Asia/Seoul /etc/localtime
sudo timedatectl set-timezone Asia/Seoul
timedatectl

# Java 8 Install		
sudo apt-get update
sudo apt install openjdk-8-jdk -y
		
# Spark Install
wget https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
tar -xvf spark-3.5.3-bin-hadoop3.tgz
sudo mv spark-3.5.3-bin-hadoop3 /opt/spark

# Delta Lake 라이브러리	
wget https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.2.0/delta-spark_2.12-3.2.0.jar -P /opt/spark/jars;
wget https://repo1.maven.org/maven2/io/delta/delta-storage/3.2.0/delta-storage-3.2.0.jar -P /opt/spark/jars

# Hadoop의 S3A 관련 라이브러리	
wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/3.3.4/hadoop-common-3.3.4.jar -P /opt/spark/jars;
wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar -P /opt/spark/jars;
wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-core/3.3.4/hadoop-mapreduce-client-core-3.3.4.jar -P /opt/spark/jars;
wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar -P /opt/spark/jars;
wget https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.Final/wildfly-openssl-1.0.7.Final.jar -P /opt/spark/jars

#### 4. Python 설치

In [ ]:
# 시스템 환경에 Python(3.12.3) 설치
sudo apt update
sudo apt install python3 python3-pip python3-venv python-is-python3
sudo apt install libpq-dev python3-dev build-essential
sudo apt install -y curl nodejs npm
python --version

# 가상환경(base) 생성
#    airflow(2.10.2) + mlflow(2.19.0) + sqlmesh(0.140.1) +
#    jupyterhub(5.2.1) + dash(2.9.3) + pyspark(3.5.4) + deltaspark(3.3.0)
sudo python3 -m venv /usr/local/venv/base
sudo chown -R aibiz:aibiz /usr/local/venv
/usr/local/venv/base/bin/pip install -r ./python/requirements_base.txt
source /usr/local/venv/base/bin/activate

# (참고) 버전검사
/usr/local/venv/base/bin/airflow version
/usr/local/venv/base/bin/mlflow --version
/usr/local/venv/base/bin/sqlmesh --version
/usr/local/venv/base/bin/jupyterhub --version
/usr/local/venv/base/bin/pip show dash
/usr/local/venv/base/bin/pip show pyspark
/usr/local/venv/base/bin/pip show delta-spark

# 커널설치 (bash)
/usr/local/venv/base/bin/python3 -m bash_kernel.install --prefix=/usr/local/venv/base

#### 5. 시스템 운영

In [ ]:
# docker build
cd ~/setup
make all-build

# certbot 기동(인증서 가져오기: 3개월에 1회만 실행하면 됨)
make certbot-up
# 실행하고 나서 잠시 기다리면
#   ~/setup/volumes/certbot/conf/live폴더 아래에 rnd7.ai-biz.net 폴더와 파일이 생성됨.
# certbot 종료
make certbot-down

# 권한 지정
sudo chown -R aibiz:aibiz /data/volumes

# nginx 확인
make nginx-up

# keycloak 확인
make postgres-up
make keycloak-up
# keycloak의 client의 주소를 수정(rnd6->sam1)
# realm(dutchboy) > Clients 각각으로 들어가서 URL부분 모두 수정

make portainer-up
# portainer의 SSO 주소를 수정(rnd6->sam1)
# Local 사용자로 로그인 > Settings>Authentication 메뉴 (6곳 수정)

# 전체 서비스 기동
make all-up

# 전체 서비스 종료
make all-down

#### 6. minio 백업 및 복원 가이드

In [ ]:
# mc복사 방법
# 아래 프로그램 다운로드 받아서 윈도우 경로에 복사
https://dl.min.io/client/mc/release/windows-amd64/mc.exe

# 서버 접속(별명지정)
mc.exe alias set myminio http://rnd7.ai-biz.net:7078 aibiz aibiz12#$

# 폴더 목록 확인
mc ls myminio

# 원격에서 폴더 전체 복사 방법
mc cp -r myminio/mlflow.db/ ./test


# node.js 설치 for jupyterhub
sudo apt-get install -y nodejs